In [11]:
from sklearn.naive_bayes import GaussianNB
import sklearn.svm
from sklearn.pipeline import Pipeline
import sys
import argparse
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
import string
from nltk.tokenize import word_tokenize
import nltk
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, GridSearchCV


In [4]:
engine = create_engine('sqlite:///{}'.format('C:\\Users\\Mo\\Desktop\\Udacity\\disaster_response_pipeline_project\\pycharm\\data\\database.db'))
df = pd.read_sql_table('data', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original'], axis = 1)
category_names = list(Y.columns)

In [5]:
def tokenize(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().strip()
    text = word_tokenize(text)
    # text = list(set(words) - set(stopwords.words("english")))
    return text

In [29]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

    parameters = [{"clf": [RandomForestClassifier()],
      "clf__max_depth": list(range(1,21))
      }
     # {"clf": [sklearn.svm.LinearSVC()]
     #    # "classifier__penalty": ['l1', 'l2'],
     #  # "classifier__C": np.logspace(0, 4, 10)
     #  },
     # {"clf": [GaussianNB()]
     #    # "classifier__penalty": ['l1', 'l2'],
     #    # "classifier__C": np.logspace(0, 4, 10)
     #  }
     #    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
     # 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
     # 'features__text_pipeline__vect__max_features': (None, 5000, 10000),
     # 'features__text_pipeline__tfidf__use_idf': (True, False),
     # 'clf__n_estimators': [50, 100, 200],
     # 'clf__min_samples_split': [2, 3, 4],
     # 'features__transformer_weights': (
     #     {'text_pipeline': 1, 'starting_verb': 0.5},
     #     {'text_pipeline': 0.5, 'starting_verb': 1},
     #     {'text_pipeline': 0.8, 'starting_verb': 1},
     # )
     ]
    # param_dist = {"max_depth": [3, None],
    #               "max_features": sp_randint(1, 11),
    #               "min_samples_split": sp_randint(2, 11),
    #               "bootstrap": [True, False],
    #               "criterion": ["gini", "entropy"]}

    n_iter_search = 20

    rskf = RepeatedStratifiedKFold(
        n_splits=10,
        n_repeats=2,
        random_state=1337)

   cv = RandomizedSearchCV(
        pipeline,
        param_grid=parameters,
        n_iter=n_iter_search,
        cv=rskf)

    final_model = cv.best_estimator_


    return cv.fit(X, Y["genre"])

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 44)

In [17]:
fit = build_model()



C:\Users\Mo\Anaconda3\envs\disaster_response_pipeline\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [23]:
fit.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                ('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=3,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min

In [27]:
fit.score(X,)

ValueError: Expected array-like (array or non-string sequence), got None